# Run inference on time to merge model trained previously


## What we did previously

In the previous [notebook](./03_model_training.ipynb) we trained machine learning models to classify a PR's `time_to_merge` into one of the 10 bins (or "classes"). We then deployed the model with the highest f1-score as a service using the model saved in s3.

## In this step

In this notebook, we are going to fetch the model that we previously trained, saved and stored in s3. We will send a payload to this model and see how it performs on the test data.
# Time to Merge Prediction Inference Service

In the previous notebook, we explored some basic machine learning models for predicting time to merge of a PR.

In [1]:
import os
import sys
import gzip
import json
import boto3
import datetime
import requests
from dotenv import load_dotenv, find_dotenv
from io import BytesIO

import joblib

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report

load_dotenv(find_dotenv(), override=True)

import warnings
warnings.filterwarnings('ignore') 

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
class CephCommunication:
    """
    Class to establish communication with a ceph s3 bucket.
    It connects with the bucket and provides methods to read and write data in the parquet format.
    """

    def __init__(
        self, s3_endpoint_url, aws_access_key_id, aws_secret_access_key, s3_bucket
    ):
        self.s3_endpoint_url = s3_endpoint_url
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
        self.s3_resource = boto3.resource(
            "s3",
            endpoint_url=self.s3_endpoint_url,
            aws_access_key_id=self.aws_access_key_id,
            aws_secret_access_key=self.aws_secret_access_key,
        )
        self.bucket = s3_bucket
        ## Todo: Add try catch

    def upload_to_ceph(self, dataframe, s3_path, filename):
        """
        This helper function takes as input the data frame to be uploaded, and the output filename.
        It then saves the data frame in the defined ceph bucket.
        """
        parquet_buffer = BytesIO()
        dataframe.to_parquet(parquet_buffer)
        s3_obj = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        status = s3_obj.put(Body=parquet_buffer.getvalue())
        return status

    def read_from_ceph(self, s3_path, filename):
        """
        Helper function to read from ceph and see if the saved data is correct.
        """
        buffer = BytesIO()
        s3_object = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        s3_object.download_fileobj(buffer)
        df_temp = pd.read_parquet(buffer)
        return df_temp


def save_to_disk(dataframe, path, filename):
    """
    Helper function to save the dataframe
    as a parquet file to disk.
    """
    dataset_base_path = Path(path)
    dataset_base_path.mkdir(parents=True, exist_ok=True)
    dataframe.to_parquet(f"{path}/{filename}")
    return True

In [3]:
## CEPH Bucket variables
## Create a .env file on your local with the correct configs,

ORG = os.getenv("GITHUB_ORG")
REPO = os.getenv("GITHUB_REPO")

## S3 bucket credentials
s3_endpoint_url = os.getenv("S3_ENDPOINT_URL")
s3_access_key = os.getenv("AWS_ACCESS_KEY_ID")
s3_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_bucket = os.getenv("S3_BUCKET")

s3_input_data_path = os.getenv("CEPH_BUCKET_PREFIX")

REMOTE = os.getenv("REMOTE")
RAW_DATA_PATH = os.path.join(
    s3_input_data_path, "srcopsmetrics/bot_knowledge", ORG, REPO, "PullRequest.json"
)

In [4]:
output = []
local_input_data_path = "../../../data/raw/GitHub/PullRequest.json.gz"
if REMOTE:
    print("getting dataset from ceph")
    s3 = boto3.resource(
        "s3",
        endpoint_url=s3_endpoint_url,
        aws_access_key_id=s3_access_key,
        aws_secret_access_key=s3_secret_key,
    )
    content = s3.Object(s3_bucket, RAW_DATA_PATH)
    file = content.get()["Body"].read().decode("utf-8")

    prs = json.loads(file)

    for pr in prs.splitlines():
        output.append(json.loads(pr))

else:
    print("getting dataset from local")
    with gzip.open(local_input_data_path, "r") as f:
        prs = json.loads(f.read().decode("utf-8"))


pr_df = pd.DataFrame(output)

getting dataset from ceph


In [5]:
# github pr dataset collected using thoth's mi-scheduler
pr_df.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,Fixed argparse,## Related Issues and Dependencies\r\n\r\nhttp...,S,Gkrumbach07,1661803585,1661812038,sesheta,1.661812e+09,sesheta,6,1,"{'Gkrumbach07': 1, 'sesheta': 65}",{},"[approved, size/S]","[d3d60750ec0ebbc2169b07ee68a340cdd4b8e1b4, c77...",[jupyterlab_requirements/dependency_management...,NaN,NaN,638
1,Use the pyproject.toml for mypy conf and pre-c...,Use the pyproject.toml for mypy conf and pre-c...,M,harshad16,1660662213,1660663187,harshad16,1.660663e+09,harshad16,1,4,"{'harshad16': 4, 'sesheta': 65}",{},"[approved, size/M]",[64cc9192e437645f581d43ccac3219d780300c0b],"[.pre-commit-config.yaml, .prow.yaml, mypy.ini...",NaN,NaN,637
2,fixed api bugs,## Related Issues and Dependencies\r\nhttps://...,L,Gkrumbach07,1660660810,1660671365,sesheta,1.660671e+09,sesheta,2,3,"{'sesheta': 175, 'Gkrumbach07': 1}","{'1074336988': {'author': 'harshad16', 'words_...","[approved, size/L, lgtm]","[d3d60750ec0ebbc2169b07ee68a340cdd4b8e1b4, c77...",[src/components/dependencyManagementConstraint...,1.660664e+09,NaN,636
3,Release of version 0.16.2,"Hey, @harshad16!\n\nOpening this PR to fix the...",XS,khebhut[bot],1656682742,1656682893,harshad16,1.656683e+09,harshad16,1,2,{'sesheta': 257},"{'1026234712': {'author': 'sefkhet-abwy[bot]',...","[bot, approved, size/XS, needs-ok-to-test, ok-...",[75403007e7be2dcbf95e3a69040c7470f664703c],"[CHANGELOG.md, jupyterlab_requirements/__init_...",1.656683e+09,1.656683e+09,635
4,HouseKeeping: Updated OWNERS and pre-commit ch...,HouseKeeping: Updated OWNERS and pre-commit ch...,L,harshad16,1656608952,1656682684,harshad16,1.656683e+09,harshad16,1,16,"{'sesheta': 192, 'harshad16': 12}",{},[size/L],[c2a4902aed5e951c3946a072a451f56c3917400b],"[.pre-commit-config.yaml, .prow.yaml, OWNERS, ...",NaN,NaN,633


In [6]:
interval = (pr_df["merged_at"] - pr_df["created_at"]).astype("float")
interval = interval.dropna()
interval

0       8453.0
1        974.0
2      10555.0
3        151.0
4      73732.0
        ...   
300        7.0
301       15.0
302       22.0
303       48.0
304       17.0
Length: 293, dtype: float64

In [7]:
n_buckets = 10

quantiles = interval.quantile(q=np.arange(0, 1 + 1e-100, 1 / n_buckets))
quantiles

0.0        6.0
0.1       38.4
0.2       65.8
0.3      107.0
0.4      244.8
0.5      402.0
0.6      592.2
0.7     1333.4
0.8     6703.6
0.9    55766.2
dtype: float64

In [8]:
quantiles / 3600

0.0     0.001667
0.1     0.010667
0.2     0.018278
0.3     0.029722
0.4     0.068000
0.5     0.111667
0.6     0.164500
0.7     0.370389
0.8     1.862111
0.9    15.490611
dtype: float64

In [9]:
time_intervals = quantiles / 3600

In [10]:
# remove PRs from train/test which are still open
pr_df = pr_df[pr_df["closed_at"].notna()]
pr_df = pr_df[pr_df["merged_at"].notna()]

In [11]:
pr_df["created_at"] = pr_df["created_at"].apply(
    lambda x: int(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["closed_at"] = pr_df["closed_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["merged_at"] = pr_df["merged_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)

In [12]:
TEST_DATA_PATH = os.path.join(s3_input_data_path, ORG, REPO, "test-data")

# read processed and split data created for train/test in the model training notebook
if REMOTE:
    cc = CephCommunication(s3_endpoint_url, s3_access_key, s3_secret_key, s3_bucket)
    X_test = cc.read_from_ceph(TEST_DATA_PATH, "X_test.parquet")
    y_test = cc.read_from_ceph(TEST_DATA_PATH, "y_test.parquet")

else:
    print(
        "The X_test.parquet and y_test.parquet files are not included in the github repo."
    )
    print(
        "Please set REMOTE=1 in the .env file and read this data from the S3 bucket instead."
    )

In [13]:
X_test

,size,created_at_day,created_at_month,created_at_weekday,created_at_hour,changed_files_number,body_size,commits_number,filetype_.py,filetype_.ts,...,title_wordcount_utils,title_wordcount_version,title_wordcount_video,title_wordcount_virtual,title_wordcount_wants,title_wordcount_way,title_wordcount_wip,title_wordcount_working,title_wordcount_yaml,title_wordcount_yarn
85,0.0,10.0,8.0,1.0,12.0,2.0,25.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269,0.0,19.0,1.0,1.0,9.0,1.0,34.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,2.0,4.0,11.0,3.0,10.0,5.0,11.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179,1.0,18.0,3.0,3.0,12.0,1.0,4.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,3.0,3.0,6.0,3.0,13.0,1.0,56.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,0.0,11.0,6.0,4.0,8.0,2.0,35.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
234,1.0,16.0,2.0,1.0,18.0,1.0,53.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,2.0,23.0,6.0,2.0,9.0,1.0,53.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,17.0,2.0,3.0,18.0,4.0,4.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,3.0,9.0,8.0,0.0,21.0,2.0,7.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
y_test

,ttm_class
85,2
269,9
46,7
179,0
146,4
127,4
234,0
120,6
9,9
91,0


In [15]:
sample_payload = pr_df.reindex(X_test.index)

In [16]:
sample_payload

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
85,Release of version 0.10.3,"Hey, @pacospace!\n\nOpening this PR to fix the...",XS,khebhut[bot],1,1.628597,pacospace,1.628597,pacospace,1,2,{},"{'726360985': {'author': 'sefkhet-abwy[bot]', ...","[bot, approved, ok-to-test]",[ec5319d96f55bcb3c308db92884c0519469ceeb4],"[CHANGELOG.md, jupyterlab_requirements/__init_...",1.628598e+09,1.628598e+09,381
269,Capture output to show info,Signed-off-by: Francesco Murdaca <fmurdaca@red...,XS,pacospace,1,1.611167,sesheta,1.611167,sesheta,1,1,"{'pacospace': 2, 'sesheta': 65}",{},"[approved, size/XS]",[cdc0d59bf5cd90cd8964d5ed891608e6204317cd],[jupyterlab_requirements/dependency_management...,NaN,NaN,78
46,Add force flag for set-kernel command,Signed-off-by: Francesco Murdaca <fmurdaca@red...,M,pacospace,1,1.636023,sesheta,1.636023,sesheta,1,5,"{'pacospace': 1, 'sesheta': 65}",{},"[approved, size/M]",[c6864b3a8d5c5898155a8acc0abebbcd68704900],"[docs/source/horus-cli.md, docs/source/horus-m...",NaN,NaN,563
179,Fix name image,Signed-off-by: Francesco Murdaca <fmurdaca@red...,S,pacospace,1,1.616070,pacospace,1.616070,pacospace,1,1,{'sesheta': 85},{},[size/S],[7117101bf4203a9fc788f8fab4152d6381f2867c],[README.md],NaN,NaN,223
146,Automatic update of dependencies by Kebechet,Kebechet has updated the depedencies to the la...,L,khebhut[bot],1,1.622726,sesheta,1.622726,sesheta,1,1,"{'sesheta': 190, 'fridex': 1}","{'675266567': {'author': 'sefkhet-abwy[bot]', ...","[bot, approved, size/L, ok-to-test]",[0cda4f8e3240b5ab260e6cf812b0a1887cdab9f1],[Pipfile.lock],1.622726e+09,1.622726e+09,281
127,Release of version 0.7.2,"Hey, @pacospace!\n\nOpening this PR to fix the...",XS,khebhut[bot],1,1.623399,sesheta,1.623399,sesheta,1,2,{'sesheta': 327},"{'681561908': {'author': 'sefkhet-abwy[bot]', ...","[bot, approved, size/XS, needs-ok-to-test, ok-...",[4ba32e1d160ad17906dae785963394f2b01fa4aa],"[CHANGELOG.md, jupyterlab_requirements/__init_...",1.623399e+09,1.623399e+09,308
234,Automatic update of dependencies by Kebechet,Kebechet has updated the depedencies to the la...,S,khebhut[bot],1,1.613499,sesheta,1.613499,sesheta,1,1,{'sesheta': 140},"{'591502077': {'author': 'sefkhet-abwy[bot]', ...","[bot, approved, size/S]",[32c3445a207314ad40f3054c6c1cffc50cd8f42a],[Pipfile.lock],1.613500e+09,1.613500e+09,127
120,Automatic update of dependencies by Kebechet,Kebechet has updated the depedencies to the la...,M,khebhut[bot],1,1.624441,sesheta,1.624441,sesheta,1,1,{'sesheta': 257},"{'690419952': {'author': 'sefkhet-abwy[bot]', ...","[bot, approved, size/M, needs-ok-to-test, ok-t...",[938639390d80dd54af67f5ed3e7169a6017a6565],[Pipfile.lock],1.624440e+09,1.624440e+09,318
9,Fix creation of Pipfile source section,Signed-off-by: Francesco Murdaca <fmurdaca@red...,S,pacospace,1,1.645435,sesheta,1.645435,sesheta,1,4,"{'pacospace': 1, 'sesheta': 68}","{'886407316': {'author': 'Gkrumbach07', 'words...","[approved, size/S]",[4c51f61d03c75fa01322e977e1cdf7cb2be39b46],"[src/components/dependencyManagementUI.tsx, sr...",1.645124e+09,1.645124e+09,622
91,Fix/missing path,## Related Issues and Dependencies\r\n\r\nFixe...,L,pacospace,1,1.628543,pacospace,1.628543,pacospace,2,2,{},{},[],"[fbf340e923af4aff3b0c759d950b6cc6d1a72105, 962...","[README.md, jupyterlab_requirements/dependency...",NaN,NaN,370


In [17]:
sample_payload.dtypes

title                    object
body                     object
size                     object
created_by               object
created_at                int64
closed_at               float64
closed_by                object
merged_at               float64
merged_by                object
commits_number            int64
changed_files_number      int64
interactions             object
reviews                  object
labels                   object
commits                  object
changed_files            object
first_review_at         float64
first_approve_at        float64
id                       object
dtype: object

In [18]:
## read model
MODEL_KEY = os.path.join(s3_input_data_path, ORG, REPO, "ttm-model")
MODEL_FILENAME = "model.joblib"


s3_resource = boto3.resource(
    "s3",
    endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
)

buffer = BytesIO()
s3_object = s3_resource.Object(s3_bucket, f"{MODEL_KEY}/{MODEL_FILENAME}")
s3_object.download_fileobj(buffer)
model = joblib.load(buffer)
model

Pipeline(steps=[('extract_features',
                 ColumnTransformer(transformers=[('pr_size',
                                                  OrdinalEncoder(categories=[['XS',
                                                                              'S',
                                                                              'M',
                                                                              'L',
                                                                              'XL',
                                                                              'XXL']]),
                                                  ['size']),
                                                 ('created_at_details',
                                                  DateTimeDetailsTransformer(),
                                                  ['created_at']),
                                                 ('n_changed_files',
                                                  NumChangedFilesTransformer(),
                                                  ['changed_files_number']),
                                                 ('body_size',
                                                  StringLenTransformer(),
                                                  ['body']),
                                                 ('n_commits', 'p...
                                                                                   'aicoe',
                                                                                   'analysis',
                                                                                   'api',
                                                                                   'argparse',
                                                                                   'asking',
                                                                                   'automatic',
                                                                                   'automatically',
                                                                                   'backend',
                                                                                   'banners',
                                                                                   'better',
                                                                                   'bugs',
                                                                                   'build',
                                                                                   'cachedir',
                                                                                   'capabilities',
                                                                                   'capture',
                                                                                   'cells',
                                                                                   'change',
                                                                                   'changes',
                                                                                   'check=true',
                                                                                   'clean',
                                                                                   'cli',
                                                                                   'code', ...]),
                                                  ['title'])])),
                ('scale', PowerTransformer()),
                ('rf',
                 RandomForestClassifier(max_features=0.75, n_estimators=200,
                                        n_jobs=-1, random_state=42))])

In [19]:
# Test model on the dataset
preds = model.predict(sample_payload)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.50      0.20      0.29         5
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         7
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         8
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         5
           9       0.19      0.86      0.31         7

    accuracy                           0.12        59
   macro avg       0.07      0.11      0.06        59
weighted avg       0.06      0.12      0.06        59



# Conclusion

This notebook fetches the saved model from s3 and sends a payload to see how the model is performing on this new data. Additionally, we see that the evaluation scores in the classification report match the ones we saw in the training notebook. So, great, looks like our model are working as expected, and are ready to predict some times to merge for GitHub PRs! 